In [2]:
import os,cv2,glob


In [5]:
images = [] 
labels = []

dict_labels={"Cat":0 , "Dog":1}
size = (40,40)

for folders in glob.glob("./kagglecatsanddogs_5340/PetImages/*"):
    print (folders,"image loading")
    for filename in os.listdir(folders):
        label=folders.split("/")[-1]
        #print (label)
        try:
            img=cv2.imread(os.path.join(folders,filename))
            if img is not None:
                img = cv2.resize(img,dsize=size)
                images.append(img)
                labels.append(dict_labels[label])
        except:
            print (os.path.join(folders,filename),"can't load")
            pass

./kagglecatsanddogs_5340/PetImages/Cat image loading


Corrupt JPEG data: 214 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1153 extraneous bytes before marker 0xd9
Corrupt JPEG data: 99 extraneous bytes before marker 0xd9
Corrupt JPEG data: 128 extraneous bytes before marker 0xd9
Corrupt JPEG data: 239 extraneous bytes before marker 0xd9


./kagglecatsanddogs_5340/PetImages/Dog image loading


Corrupt JPEG data: 65 extraneous bytes before marker 0xd9
Corrupt JPEG data: 226 extraneous bytes before marker 0xd9
Corrupt JPEG data: 162 extraneous bytes before marker 0xd9
Corrupt JPEG data: 2230 extraneous bytes before marker 0xd9
Corrupt JPEG data: 254 extraneous bytes before marker 0xd9
Corrupt JPEG data: 399 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1403 extraneous bytes before marker 0xd9


In [6]:
print (len(images),len(labels))

24946 24946


In [7]:
from sklearn.model_selection import train_test_split

train_feature,test_feature,train_label,test_label = train_test_split(images,labels,test_size=0.2,random_state=42)

In [8]:
import numpy as np

train_feature = np.array(train_feature)
test_feature = np.array(test_feature)
train_label = np.array(train_label)
test_label = np.array(test_label)

In [11]:
print (len(train_feature),len(test_feature))
print (train_feature.shape,train_label.shape)
print (test_feature.shape,test_label.shape)

19956 4990
(19956, 40, 40, 3) (19956,)
(4990, 40, 40, 3) (4990,)


In [12]:
imagesavepath="Cat_Dog_Dataset/"
if not os.path.exists(imagesavepath):
    os.mkdir(imagesavepath)

np.save(imagesavepath+"train_feature.npy",train_feature)
np.save(imagesavepath+"test_feature.npy",test_feature)
np.save(imagesavepath+"train_label.npy",train_label)
np.save(imagesavepath+"test_label.npy",test_label)


In [13]:
train_feature_vector = train_feature.reshape(len(train_feature),40,40,3).astype('float32')
test_feature_vector = test_feature.reshape(len(test_feature),40,40,3).astype('float32')

In [14]:
train_feature_normalize = train_feature_vector/255
test_feature_normalize = test_feature_vector/255

In [16]:
from tensorflow.python.keras.utils.np_utils import to_categorical

train_label_onehot = to_categorical(train_label)
test_label_onehot = to_categorical(test_label)

In [17]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dropout,Flatten,Dense

In [18]:
model = Sequential()

In [19]:
model.add(Conv2D(filters=10,kernel_size=(5,5),padding='same',input_shape=(40,40,3),activation='relu'))

In [20]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [21]:
model.add(Dropout(0.1))

In [22]:
model.add(Conv2D(filters=20,kernel_size=(5,5),padding='same',activation='relu'))

In [23]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [24]:
model.add(Dropout(0.2))

In [25]:
model.add(Flatten())

In [26]:
model.add(Dense(units=512,activation='relu'))

In [28]:
model.add(Dense(units=2,activation='softmax'))

In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 40, 40, 10)        760       
                                                                 
 max_pooling2d (MaxPooling2  (None, 20, 20, 10)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 20, 20, 10)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 20, 20, 20)        5020      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 10, 10, 20)        0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 10, 10, 20)        0

In [30]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [31]:
train_history= model.fit(x=train_feature_normalize,y=train_label_onehot,validation_split=0.2,epochs=10,batch_size=200,verbose=2)

Epoch 1/10
80/80 - 26s - loss: 0.7051 - accuracy: 0.5586 - val_loss: 0.6833 - val_accuracy: 0.5616 - 26s/epoch - 328ms/step
Epoch 2/10
80/80 - 23s - loss: 0.6478 - accuracy: 0.6240 - val_loss: 0.6523 - val_accuracy: 0.6075 - 23s/epoch - 292ms/step
Epoch 3/10
80/80 - 24s - loss: 0.6051 - accuracy: 0.6685 - val_loss: 0.5906 - val_accuracy: 0.6759 - 24s/epoch - 296ms/step
Epoch 4/10
80/80 - 23s - loss: 0.5711 - accuracy: 0.6984 - val_loss: 0.5719 - val_accuracy: 0.7039 - 23s/epoch - 290ms/step
Epoch 5/10
80/80 - 23s - loss: 0.5462 - accuracy: 0.7185 - val_loss: 0.5447 - val_accuracy: 0.7229 - 23s/epoch - 282ms/step
Epoch 6/10
80/80 - 22s - loss: 0.5188 - accuracy: 0.7425 - val_loss: 0.5762 - val_accuracy: 0.6994 - 22s/epoch - 279ms/step
Epoch 7/10
80/80 - 22s - loss: 0.5011 - accuracy: 0.7551 - val_loss: 0.5272 - val_accuracy: 0.7255 - 22s/epoch - 277ms/step
Epoch 8/10
80/80 - 23s - loss: 0.4756 - accuracy: 0.7711 - val_loss: 0.5139 - val_accuracy: 0.7442 - 23s/epoch - 286ms/step
Epoch 9/

In [33]:
scores = model.evaluate(test_feature_normalize,test_label_onehot)
print ("scores = ", scores[1])

156/156 [==============================] - 2s 15ms/step - loss: 0.4938 - accuracy: 0.7659
scores =  0.7659318447113037


In [34]:
prediction = model.predict(test_feature_normalize)
prediction = np.argmax(prediction,axis=1)

156/156 [==============================] - 4s 23ms/step


In [35]:
print (test_label[0:10],prediction[0:10])

[1 0 0 0 0 1 1 1 1 0] [1 1 0 1 0 1 0 1 1 0]


In [36]:
model.save('cat_dog_CNN_model.h5')

/Users/ojy/anaconda3/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
